In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

In [ ]:
def getAllCours():
    URL_ROOT = 'https://edu.epfl.ch/'
    shs = ['https://edu.epfl.ch/studyplan/fr/bachelor/programme-sciences-humaines-et-sociales/', 'https://edu.epfl.ch/studyplan/fr/master/programme-sciences-humaines-et-sociales/']
    page = requests.get(URL_ROOT)
    soup = BeautifulSoup(page.content, "html.parser")
    cards = soup.findAll("div", class_="card-title")
    annees = [card.find('a').get('href') for card in cards]
    courses = []
    for annee in annees:
        page = requests.get(URL_ROOT + annee)
        soup = BeautifulSoup(page.content, "html.parser")
        sections = [x.get('href') for x in soup.find('main').find('ul').findAll('a')]
        for section in sections:
            page = requests.get(URL_ROOT + section)
            soup = BeautifulSoup(page.content, "html.parser")
            for cours in soup.find('main').findAll('div', class_="cours-name"):
                if cours.find('a') != None:
                    courses.append(cours.find('a').get('href').split('/').pop())
    courses.remove('programme-sciences-humaines-et-sociales')
    for url in shs:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        for cours in soup.findAll("div", class_="cours-name"):
            if cours.find('a') != None:
                courses.append(cours.find('a').get('href').split('/').pop())
                
    return courses

In [150]:
def parseCours(url):
    page = requests.get(url)
    if (page.status_code == 404):
        print(url)
        return
        
    soup = BeautifulSoup(page.content, "html.parser")
    
    title = soup.find('main').find('h1').text
    if (soup.find('div', class_="course-summary") == None):
        print(url)
    code = soup.find('div', class_="course-summary").findAll('p')[0].text.split('/')[0].strip()
    credits = int(re.findall(r'\d+', soup.find('div', class_="course-summary").findAll('p')[0].text.split('/')[1])[0])
    teachers = [(x.text, x.get('href')) for x in soup.find('div', class_="course-summary").findAll('p')[1].findAll('a')]
    

    schedule = dict()
    if (soup.find("table", class_="semaineDeRef") != None):
        rows = soup.find("table", class_="semaineDeRef").findAll("tr")
    
        for i, row in enumerate(rows):
            col = row.findAll('td')
            for j, col in enumerate(col):
                if (i == 0):
                    if (j > 0):
                        day = col.text
                else:
                    if (j == 0):
                        time = col.text
                    else:
                        classes = col.get('class')
                        if (classes != None and "taken" in classes):
                            if (col.get('rowspan') == None):
                                duration = 1
                            else:
                                duration = int(col.get('rowspan'))
                            classes.remove('taken')
                            if (len(classes) != 0):
                                label = classes[0]
                                rooms = [room.text for room in col.findAll('a')]
                                if (time not in schedule):
                                    schedule[time] = dict()
                                schedule[time][day] = {
                                    'label': label,
                                    'duration': duration,
                                    'rooms': rooms
                                }
    course = {
        'name': title,
        'code': code,
        'credits': credits,
        'teachers': teachers,
        'schedule': schedule
    }

    return course

In [103]:
courses_url = getAllCours()

In [153]:
URL_ROOT = 'https://edu.epfl.ch/coursebook/fr/'
courses_url = list(set(courses_url))

courses = []
for url in courses_url:
    courses.append(parseCours(URL_ROOT + url))



https://edu.epfl.ch/coursebook/fr/human-computer-interaction-CS-213
https://edu.epfl.ch/coursebook/fr/programme-sciences-humaines-et-sociales
https://edu.epfl.ch/coursebook/fr/hydrological-risks-and-structures-ENV-524


In [161]:
courses = list(filter(lambda x: x != None, courses))

[{'name': 'Machine learning', 'code': 'CS-433', 'credits': 8, 'teachers': [('Jaggi Martin', 'https://people.epfl.ch/276449?lang=fr'), ('Flammarion Nicolas Henri Bernard', 'https://people.epfl.ch/317566?lang=fr')], 'schedule': {'10-11': {'Ve': {'label': 'cours', 'duration': 2, 'rooms': ['RLC E1 240']}}, '14-15': {'Ve': {'label': 'exercice', 'duration': 2, 'rooms': ['INF2', 'INF119', 'INM202', 'INJ218', 'INR219']}}, '16-17': {'Ve': {'label': 'cours', 'duration': 2, 'rooms': ['RLC E1 240']}}}}]


In [165]:
import pickle 

with open('courses.pkl', 'wb') as f:
    pickle.dump(courses, f)

In [ ]:
# Load data (deserialize)
with open('courses.pkl', 'rb') as handle:
    courses = pickle.load(handle)